In [ ]:
# 1. Cài đặt thư viện
!pip install fastapi uvicorn pyngrok sentence-transformers nest-asyncio

import uvicorn
import nest_asyncio
from fastapi import FastAPI, Body
from sentence_transformers import SentenceTransformer
from pyngrok import ngrok
from typing import List

# 2. Load Model (Chỉ load 1 lần khi khởi động)
# Đảm bảo bạn đã bật GPU (Runtime > Change runtime type > T4 GPU) để chạy nhanh
print("Đang tải model BGE-M3...")
model = SentenceTransformer('BAAI/bge-m3')
print("Model đã sẵn sàng!!")


Đang tải model BGE-M3...


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/123 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

...

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/964 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/191 [00:00<?, ?B/s]

Model đã sẵn sàng!!


In [ ]:
# 3. Tạo App API
app = FastAPI()

@app.post("/embed")
async def create_embedding(text: str = Body(..., embed=True)):
    # Tạo vector đơn lẻ
    vector = model.encode(text)
    return {"embedding": vector.tolist()}

@app.post("/embed_batch")
async def create_embedding_batch(texts: List[str] = Body(..., embed=True)):
    # Model encode cả danh sách cùng lúc cực nhanh trên GPU
    # batch_size=32 là an toàn cho Colab, nếu tràn RAM thì giảm xuống 16
    vectors = model.encode(texts, batch_size=32)
    return {"embeddings": vectors.tolist()}

# 4. Mở đường hầm Ngrok (Public URL)
# ⚠️ CẢNH BÁO: Bạn nên vào dashboard.ngrok.com reset lại token vì token cũ đã bị lộ
# Dán token của bạn vào bên dưới:
YOUR_TOKEN = "YOUR_NGROK_TOKEN"
ngrok.set_auth_token(YOUR_TOKEN)

# Đóng các tunnel cũ nếu có để tránh lỗi
ngrok.kill()

# Mở tunnel mới
public_url = ngrok.connect(8000)
print(f"\nAPI CỦA BẠN ĐANG CHẠY TẠI: {public_url}\n")

# 5. Chạy Server (Đoạn này đã được sửa lỗi asyncio)
nest_asyncio.apply()

# Cấu hình server thủ công thay vì dùng uvicorn.run()
config = uvicorn.Config(app, port=8000)
server = uvicorn.Server(config)

# Chạy server trên loop có sẵn của Colab
await server.serve()


API CỦA BẠN ĐANG CHẠY TẠI: NgrokTunnel: "https://hypergamous-bernadine-unspitefully.ngrok-free.dev" -> "http://localhost:8000"



INFO:     Started server process [1337]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)


INFO:     42.112.96.169:0 - "POST /embed_batch HTTP/1.1" 200 OK
INFO:     42.112.96.169:0 - "POST /embed_batch HTTP/1.1" 200 OK
INFO:     42.112.96.169:0 - "POST /embed_batch HTTP/1.1" 200 OK
INFO:     42.112.96.169:0 - "POST /embed_batch HTTP/1.1" 200 OK
INFO:     42.112.96.169:0 - "POST /embed_batch HTTP/1.1" 200 OK
INFO:     42.112.96.169:0 - "POST /embed_batch HTTP/1.1" 200 OK
INFO:     42.112.96.169:0 - "POST /embed_batch HTTP/1.1" 200 OK
INFO:     42.112.96.169:0 - "POST /embed_batch HTTP/1.1" 200 OK
INFO:     42.112.96.169:0 - "POST /embed_batch HTTP/1.1" 200 OK
INFO:     42.112.96.169:0 - "POST /embed_batch HTTP/1.1" 200 OK
INFO:     42.112.96.169:0 - "POST /embed_batch HTTP/1.1" 200 OK
INFO:     42.112.96.169:0 - "POST /embed_batch HTTP/1.1" 200 OK
INFO:     42.112.96.169:0 - "POST /embed_batch HTTP/1.1" 200 OK
INFO:     42.112.96.169:0 - "POST /embed_batch HTTP/1.1" 200 OK
INFO:     42.117.243.5:0 - "POST /embed_batch HTTP/1.1" 200 OK
INFO:     42.117.243.5:0 - "POST /embed_b